In [2]:
import binance
import pandas as pd
import numpy as np
import math

In [3]:
ticker = "MANAUSDT"
client = binance.Client()

# Traemos precio actual

In [4]:
precio_actual = float(client.futures_mark_price(symbol=ticker)['markPrice'])
precio_actual

0.96778035

# Traemos el Order Book de la moneda

In [5]:
order_book = client.futures_order_book(symbol=ticker, limit=1000)
order_book

{'lastUpdateId': 1773000114889,
 'E': 1659476229320,
 'T': 1659476229313,
 'bids': [['0.9677', '3738'],
  ['0.9676', '7401'],
  ['0.9675', '4900'],
  ['0.9674', '2197'],
  ['0.9673', '4352'],
  ['0.9672', '4019'],
  ['0.9671', '7943'],
  ['0.9670', '8282'],
  ['0.9669', '4319'],
  ['0.9668', '13716'],
  ['0.9667', '2979'],
  ['0.9666', '5749'],
  ['0.9665', '1427'],
  ['0.9664', '6299'],
  ['0.9663', '6221'],
  ['0.9662', '9018'],
  ['0.9661', '9199'],
  ['0.9660', '4028'],
  ['0.9659', '2789'],
  ['0.9658', '103'],
  ['0.9657', '5741'],
  ['0.9656', '6146'],
  ['0.9655', '15934'],
  ['0.9654', '24903'],
  ['0.9653', '13465'],
  ['0.9652', '13039'],
  ['0.9651', '4135'],
  ['0.9650', '1966'],
  ['0.9649', '28594'],
  ['0.9648', '2129'],
  ['0.9647', '2585'],
  ['0.9646', '10435'],
  ['0.9645', '21302'],
  ['0.9644', '1314'],
  ['0.9643', '6401'],
  ['0.9642', '4434'],
  ['0.9641', '5022'],
  ['0.9640', '318'],
  ['0.9639', '50783'],
  ['0.9638', '14230'],
  ['0.9637', '4023'],
  ['0.96

# Extraer ventas y compras

## Ventas

- Del order book extraemos las ventas (asks) y lo pasamos a un dataframe
- Para dejarlo a cómo lo muestra el libro, las ventas las ordenamos de mayor a menor
- Para poder trabajar los datos los pasamos a números, en este caso float

In [6]:
ventas = order_book['asks']
ventas = pd.DataFrame(ventas, columns = ['precio', 'monedas']).sort_index(ascending=False)
ventas[['precio','monedas']] = ventas[['precio','monedas']].astype(float)
ventas

,precio,monedas
999,1.0957,99.0
998,1.0955,5.0
997,1.0954,94.0
996,1.0953,28.0
995,1.0951,11578.0
...,...,...
4,0.9682,7248.0
3,0.9681,8341.0
2,0.9680,1395.0
1,0.9679,358.0


## Compras

- Hacemos lo mismo que ventas, convertimos a float las columnas de precio y monedas

In [7]:
compras = order_book['bids']
compras = pd.DataFrame(compras, columns=['precio','monedas'])
compras[['precio','monedas']] = compras[['precio','monedas']].astype(float)
compras

,precio,monedas
0,0.9677,3738.0
1,0.9676,7401.0
2,0.9675,4900.0
3,0.9674,2197.0
4,0.9673,4352.0
...,...,...
995,0.8553,66.0
996,0.8552,224.0
997,0.8551,18.0
998,0.8550,183.0


# Creación de rangos

- Creamos un rango tanto para ventas como compras 
- Este rango es derivado del precio máximo y mínimo

## Obtenemos máximos y mínimos para las ventas y compras

In [8]:
ventas_max = ventas['precio'].max()
print(ventas_max)

ventas_min = ventas['precio'].min()
print(ventas_min)

1.0957
0.9678


In [9]:
compras_max = compras['precio'].max()
print(compras_max)
compras_min = compras['precio'].min()
print(compras_min)

0.9677
0.8549


- Creamos rangos con esos valores
- Para esto necesitamos crear un diccionario donde coloquemos, para cada moneda, los múltiplos para saber los incrementos del rango
- Recordando que vamos a tratar de capturar todos los rangos permitidos del libro de binance

In [10]:
dic = {
    'MANAUSDT': {
        'd': 4,
        'i1': 0.01,
        'i2': 0.001,
        'i3': 0.0001
    }
}

- Ahora obtenemos el inicio del rango , usando:

(precio min / dic['ticker'] ) -> De este número, nos interesa el entero inferior. Por lo que lo redondearemos hacia el entero inferior (math.floor())

math.floor((precio min / dic['ticker'] )) * dic['ticker'] -> Luego multplicamos por el mismo incremento

- Ahora obtenemos el final del rango , usando:

(precio max / dic['ticker'] ) -> De este número, nos interesa el entero superior. Por lo que lo redondearemos hacia el entero superior (math.ceil())

math.ceil( (precio max / dic['ticker'] ) ) * dic['ticker']

In [11]:
rango_v_i = math.floor(ventas_min / dic[ticker]['i1']) * dic[ticker]['i1']
rango_v_i
rango_v_f = math.ceil(ventas_max / dic[ticker]['i1']) * dic[ticker]['i1']
rango_v_f
rango_v = np.arange(rango_v_i, rango_v_f, dic[ticker]['i1'])

In [12]:
rango_c_i = math.floor(compras_min / dic[ticker]['i1']) * dic[ticker]['i1']
rango_c_i
rango_c_f = (math.ceil(compras_max / dic[ticker]['i1']) * dic[ticker]['i1'])
rango_c_f = rango_c_f + dic[ticker]['i1']
rango_c = np.arange(rango_c_i, rango_c_f, dic[ticker]['i1'])

In [13]:
#compras
#rango_c
compras_ag = compras.groupby(pd.qcut(compras.precio, rango_c)).sum()
#print(compras_ag['monedas'].max())

idx = compras_ag.monedas.idxmax()
idx.left
print(f'Máx compras: {compras_ag["monedas"].max()}')
print(f'En el índice: {compras_ag.monedas.idxmax()}')
print(f'Precio: ${idx.right}')
#compras_ag = compras

Máx compras: 141757.0
En el índice: (0.959, 0.96]
Precio: $0.96


In [14]:
#compras
#rango_c
ventas_ag = ventas.groupby(pd.qcut(ventas.precio, rango_c)).sum()
#print(compras_ag['monedas'].max())

idx = compras_ag.monedas.idxmax()
idx.left
print(f'Máx compras: {compras_ag["monedas"].max()}')
print(f'En el índice: {compras_ag.monedas.idxmax()}')
print(f'Precio: ${idx.right}')

Máx compras: 141757.0
En el índice: (0.959, 0.96]
Precio: $0.96


In [25]:
ventas_ag

,precio,monedas
precio,,
"(1.0730000000000002, 1.075]",10.7437,8497.0
"(1.075, 1.076]",10.7570,2278.0
"(1.076, 1.078]",10.7745,662.0
"(1.078, 1.08]",10.7919,1298.0
"(1.08, 1.082]",10.8070,3583.0
"(1.082, 1.083]",10.8225,2924.0
"(1.083, 1.084]",10.8341,1983.0
"(1.084, 1.086]",10.8493,12644.0
"(1.086, 1.087]",10.8638,1663.0


In [1]:
import asyncio
import binance

In [2]:
client = binance.Client()

In [ ]:
async def main():
    order_book = await client.futures_order_book(symbol='BTCUSDT')
    print(order_book)